In [1]:
import pickle
import numpy as np
import os
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.over_sampling import RandomOverSampler

f = open("../../dataset/sense/dict_sense-keys", 'rb')
dict_sense_keys = pickle.load(f)
f.close()

f = open("../../dataset/sense/dict_word-sense", 'rb')
dict_word_sense = pickle.load(f)
f.close()

f = open('../Glove/word_embedding_glove', 'rb')
word_embedding = pickle.load(f)
f.close()
word_embedding = word_embedding[: len(word_embedding)-1]

f = open('../Glove/vocab_glove', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"

with open('/data/aviraj/dataset/raw_preprocess_train','rb') as f:
    data=pickle.load(f)

with open('/data/aviraj/dataset/fulldata_vocab_sense','rb') as f:
    vocab_lex=pickle.load(f)

lex2id = dict((s, i) for i,s in enumerate(vocab_lex))
id2lex = dict((i, s) for i,s in enumerate(vocab_lex))

print(len(vocab_lex))
max_sent_size = 200

46


In [2]:
_pos = []
for i in range(len(data)):
    for pp in data[i][4]:
        _pos.append(pp)
        
pos_count = Counter(_pos)
pos_count = pos_count.most_common()
vocab_pos = [pp for pp, c in pos_count]
pos2id = dict((s, i) for i,s in enumerate(vocab_pos))
print(len(vocab_pos))

12


In [3]:
data_y1 = []
data_y2 = []
data_y3 = []
for i in range(len(data)):
    if (len(data[i][1])<=200):
        for j in range(len(data[i][2])):
            if data[i][2][j] is not None:
                data_y1.append(dict_sense_keys[data[i][2][j]][3])
                data_y2.append(dict_sense_keys[data[i][2][j]][4])
                data_y3.append(dict_sense_keys[data[i][2][j]][5])

sense_count1 = Counter(data_y1)
sense_count1 = sense_count1.most_common()
sense_count2 = Counter(data_y2)
sense_count4 = sense_count2.most_common(272)
sense_count2 = sense_count2.most_common(312)
sense_count3 = Counter(data_y3)
sense_count5 = sense_count3.most_common(505)
sense_count3 = sense_count3.most_common(1051)

dict_sense_count1 = dict(sense_count1)
dict_sense_count2 = dict(sense_count2)
dict_sense_count3 = dict(sense_count3)
dict_sense_count4 = dict(sense_count4)
dict_sense_count5 = dict(sense_count5)

In [4]:
print(len(sense_count1), len(sense_count2), len(sense_count3), len(sense_count4), len(sense_count5))

46 312 1051 272 505


In [5]:
data_x = []
data_pos = []
data_label1 = []
data_label2 = []
data_label3 = []
data_label4 = []
data_label5 = []

for i in range(len(data)):
    if not all(np.array(data[i][2])==None) and (len(data[i][1])<=200):
        data_label1.append([ss if ss is not None and dict_sense_keys[ss][3] in dict_sense_count1 else None for ss in data[i][2]])
        data_label2.append([ss if ss is not None and dict_sense_keys[ss][4] in dict_sense_count2 else None for ss in data[i][2]])
        data_label3.append([ss if ss is not None and dict_sense_keys[ss][5] in dict_sense_count3 else None for ss in data[i][2]])
        data_label4.append([ss if ss is not None and dict_sense_keys[ss][4] in dict_sense_count4 else None for ss in data[i][2]])
        data_label5.append([ss if ss is not None and dict_sense_keys[ss][5] in dict_sense_count5 else None for ss in data[i][2]])
        data_x.append(data[i][1])
        data_pos.append(data[i][4])

In [6]:
def data_prepare(sense_id, x, pos, y, sense_count, lex_cond=False, pos_cond=False):
    num_examples = len(x)
    
    vocab_sense = [s for s, c in sense_count]
    sense2id = dict((s, i) for i,s in enumerate(vocab_sense))
    
    xx = np.zeros([num_examples, max_sent_size], dtype=int)
    xx_mask = np.zeros([num_examples, max_sent_size], dtype=bool)
    ss_mask = np.zeros([num_examples, max_sent_size], dtype=bool)
    yy = np.zeros([num_examples,max_sent_size], dtype=int)
    y_lex = np.zeros([num_examples, max_sent_size], dtype=int)
    y_pos = np.zeros([num_examples, max_sent_size], dtype=int)
        
    for j in range(num_examples):
        for i in range(max_sent_size):
            if(i>=len(x[j])):
                break
            w = x[j][i]
            s = y[j][i]
            p = pos[j][i]
            xx[j][i] = word2id[w] if w in word2id else word2id['UNKNOWN_TOKEN']
            xx_mask[j][i] = True
            ss_mask[j][i] = True if s is not None and dict_sense_keys[s][sense_id] in vocab_sense else False
            yy[j][i] = sense2id[dict_sense_keys[s][sense_id]] if s is not None and dict_sense_keys[s][sense_id] in vocab_sense else 0
            if(lex_cond):
                y_lex[j][i] = lex2id[dict_sense_keys[s][3]] if s is not None and dict_sense_keys[s][3] in vocab_lex else len(vocab_lex)
            if(pos_cond):
                y_pos[j][i] = pos2id[p] if p in vocab_pos else len(vocab_pos)
        
    return xx, xx_mask, ss_mask, yy, y_lex, y_pos

In [7]:
data_x = np.array(data_x)
data_pos = np.array(data_pos)

def train_val_data(name, sense_id, index, split_label, data_label, sense_count, sampling_list, lex_cond=False, pos_cond=False, sampling=False):
    
    index_train, index_val, label_train_id, label_val_id = train_test_split(index, split_label, train_size=0.8, shuffle=True, stratify=split_label, random_state=0)
  
    if(sampling):
        dict_sample = dict(sampling_list)
        sm = RandomOverSampler(ratio=dict_sample)
        index_train1 = np.array(index_train).reshape(-1, 1)
        sampled_index, _ = sm.fit_sample(index_train1, label_train_id)
        count = Counter(_)
        count = count.most_common()
        sampled_index_train = np.array(sampled_index).reshape(1, -1)
        index_train = sampled_index_train[0]
    
    data_label = np.array(data_label)
    x_train = data_x[index_train]
    y_train = data_label[index_train]
    x_val = data_x[index_val]
    y_val = data_label[index_val]
    pos_train = []
    pos_val = []
    
    if(pos_cond):
        pos_train = data_pos[index_train]
        pos_val = data_pos[index_val]

    x_id_train, mask_train, sense_mask_train, y_id_train, lex_train, pos_id_train  = data_prepare(sense_id, x_train, pos_train, y_train, sense_count, lex_cond=lex_cond, pos_cond=pos_cond)
    x_id_val, mask_val, sense_mask_val, y_id_val, lex_val, pos_id_val = data_prepare(sense_id, x_val, pos_val, y_val, sense_count, lex_cond=lex_cond, pos_cond=pos_cond)

    train_data = {'x':x_id_train,'x_mask':mask_train, 'sense_mask':sense_mask_train, 'y':y_id_train, 'lex':lex_train, 'pos':pos_id_train}
    val_data = {'x':x_id_val,'x_mask':mask_val, 'sense_mask':sense_mask_val, 'y':y_id_val, 'lex':lex_val, 'pos':pos_id_val}
    
    with open('/data/aviraj/dataset/train_val_data_coarse/all_word_'+ name,'wb') as f:
        pickle.dump([train_data,val_data], f)
    
    print(len(x_id_train)+len(x_id_val))

In [8]:
split_label1 = []
split_label2 = []
split_label3 = []
split_label4 = []
split_label5 = []

index1 = []
index2 = []
index3 = []
index4 = []
index5 = []

for jj, lab in enumerate(data_label1):
    min_idx = np.argmin([dict_sense_count1[dict_sense_keys[lab[i]][3]] if lab[i] is not None else np.inf for i in range(len(lab))  ]) 
    if(lab[min_idx] is not None):
        index1.append(jj)
        split_label1.append(dict_sense_keys[lab[min_idx]][3])

for jj, lab in enumerate(data_label2):
    min_idx = np.argmin([dict_sense_count2[dict_sense_keys[lab[i]][4]] if lab[i] is not None else np.inf for i in range(len(lab))  ]) 
    if(lab[min_idx] is not None):
        index2.append(jj)
        split_label2.append(dict_sense_keys[lab[min_idx]][4])

for jj, lab in enumerate(data_label3):
    min_idx = np.argmin([dict_sense_count3[dict_sense_keys[lab[i]][5]] if lab[i] is not None else np.inf for i in range(len(lab))  ]) 
    if(lab[min_idx] is not None):
        index3.append(jj)
        split_label3.append(dict_sense_keys[lab[min_idx]][5])
            
for jj, lab in enumerate(data_label4):
    min_idx = np.argmin([dict_sense_count4[dict_sense_keys[lab[i]][4]] if lab[i] is not None else np.inf for i in range(len(lab))  ]) 
    if(lab[min_idx] is not None):
        index4.append(jj)
        split_label4.append(dict_sense_keys[lab[min_idx]][4])

for jj, lab in enumerate(data_label5):
    min_idx = np.argmin([dict_sense_count5[dict_sense_keys[lab[i]][5]] if lab[i] is not None else np.inf for i in range(len(lab))  ]) 
    if(lab[min_idx] is not None):
        index5.append(jj)
        split_label5.append(dict_sense_keys[lab[min_idx]][5])
            
print(len(split_label1))
print(len(split_label2))
print(len(split_label3))
print(len(split_label4))
print(len(split_label5))

850093
850062
850052
849793
848996


In [9]:
train_val_data('lex1', 3, index1, split_label1, data_label1, sense_count1, [], lex_cond=False, pos_cond=True)
train_val_data('lex2', 3, index2, split_label2, data_label2, sense_count1, [], lex_cond=False, pos_cond=True)
train_val_data('lex3', 3, index3, split_label3, data_label3, sense_count1, [], lex_cond=False, pos_cond=True)
train_val_data('sense1', 4, index4, split_label4, data_label4, sense_count4, [], lex_cond=True, pos_cond=True)
train_val_data('sense2', 4, index5, split_label5, data_label5, sense_count4, [], lex_cond=True, pos_cond=True)
train_val_data('full_sense', 5, index5, split_label5, data_label5, sense_count5, [], lex_cond=True, pos_cond=True)

/users/btech/aviraj/cs771/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


850093
850062
850052
849793
848996
848996


In [10]:
sampled_sense_count1 = [('1:19', 10000),
 ('1:17', 10000),
 ('2:34', 10000),
 ('2:33', 10000),
 ('1:27', 10000),
 ('2:37', 8000),
 ('1:24', 8000),
 ('1:08', 8000),
 ('1:12', 7000),
 ('1:22', 5000),
 ('2:29', 5000),
 ('1:05', 3000),
 ('1:16', 3000),
 ('1:25', 3000),
 ('1:20', 3000),
 ('1:13', 2000),
 ('2:43', 1100),
 ('3:44', 1000)]

In [11]:
sampled_sense_count2= []
for s, c in sense_count2[260:]:
    sampled_sense_count2.append((s, 500))
for s, c in sense_count2[180:260]:
    sampled_sense_count2.append((s, 2000))
for s, c in sense_count2[140:180]:
    sampled_sense_count2.append((s, 5000))
for s, c in sense_count2[75:140]:
    sampled_sense_count2.append((s, 8000))
for s, c in sense_count2[25:75]:
    sampled_sense_count2.append((s, 12000))

In [12]:
sampled_sense_count3= []
for s, c in sense_count3[400:]:
    sampled_sense_count3.append((s, 500))
for s, c in sense_count3[200:400]:
    sampled_sense_count3.append((s, 2000))
for s, c in sense_count3[100:200]:
    sampled_sense_count3.append((s, 5000))
for s, c in sense_count3[70:100]:
    sampled_sense_count3.append((s, 8000))
for s, c in sense_count3[25:70]:
    sampled_sense_count3.append((s, 12000))

In [13]:
sampled_sense_count4= []
for s, c in sense_count4[260:]:
    sampled_sense_count4.append((s, 500))
for s, c in sense_count4[180:260]:
    sampled_sense_count4.append((s, 2000))
for s, c in sense_count4[140:180]:
    sampled_sense_count4.append((s, 5000))
for s, c in sense_count4[75:140]:
    sampled_sense_count4.append((s, 8000))
for s, c in sense_count4[25:75]:
    sampled_sense_count4.append((s, 12000))

In [14]:
sampled_sense_count5= []
for s, c in sense_count5[400:]:
    sampled_sense_count5.append((s, 500))
for s, c in sense_count5[200:400]:
    sampled_sense_count5.append((s, 2000))
for s, c in sense_count5[100:200]:
    sampled_sense_count5.append((s, 5000))
for s, c in sense_count5[70:100]:
    sampled_sense_count5.append((s, 8000))
for s, c in sense_count5[25:70]:
    sampled_sense_count5.append((s, 12000))

In [15]:
train_val_data('lex1_sampled', 3, index1, split_label1, data_label1, sense_count1, sampled_sense_count1, lex_cond=False, pos_cond=True, sampling=True)
train_val_data('lex2_sampled', 3, index2, split_label2, data_label2, sense_count1, sampled_sense_count2, lex_cond=False, pos_cond=True, sampling=True)
train_val_data('lex3_sampled', 3, index3, split_label3, data_label3, sense_count1, sampled_sense_count3, lex_cond=False, pos_cond=True, sampling=True)
train_val_data('sense1_sampled', 4, index4, split_label4, data_label4, sense_count4, sampled_sense_count4, lex_cond=True, pos_cond=True, sampling=True)
train_val_data('sense2_sampled', 4, index5, split_label5, data_label5, sense_count4, sampled_sense_count5, lex_cond=True, pos_cond=True, sampling=True)
train_val_data('full_sense_sampled', 5, index5, split_label5, data_label5, sense_count5, sampled_sense_count5, lex_cond=True, pos_cond=True, sampling=True)

/users/btech/aviraj/cs771/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


911174
2061567
2512876
2041581
2239996
2239996
